## Task 01
#### Create a Dummy Data Generator in Python

In [0]:
import time
import random
from azure.eventhub import EventHubProducerClient, EventData

def generate_data():
    return random.randint(50, 100)

def send_data_to_event_hub():
    connection_str = '<EVENT_HUB_CONNECTION_STRING>'
    eventhub_name = '<YOUR_EVENT_HUB_NAME>'

    producer = EventHubProducerClient.from_connection_string(conn_str=connection_str, eventhub_name=eventhub_name)

    while True:
        number = generate_data()
        event_data_batch = producer.create_batch()
        event_data_batch.add(EventData(str(number)))
        producer.send_batch(event_data_batch)
        print(f"Sent: {number}")
        time.sleep(2)

    producer.close()

send_data_to_event_hub()

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-4486013845979394>, line 24
     20         time.sleep(2)
     22     producer.close()
---> 24 send_data_to_event_hub()

File <command-4486013845979394>, line 12, in send_data_to_event_hub()
      9 connection_str = 'EVENT_HUB_CONNECTION_STRING'
     10 eventhub_name = 'YOUR_EVENT_HUB_NAME'
---> 12 producer = EventHubProducerClient.from_connection_string(conn_str=connection_str, eventhub_name=eventhub_name)
     14 while True:
     15     number = generate_data()

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-c0e2ccb6-1b8d-494a-9fd0-7a3166f4fddd/lib/python3.11/site-packages/azure/eventhub/_producer_client.py:571, in EventHubProducerClient.from_connection_string(cls, conn_str, eventhub_name, buffered_mode, buffer_concurrency, on_success, on_error, max_buffer_length, max_wait_time, logging_enable, http_proxy, auth_timeout, 

## Task 02
#### Create an Event Hub
  1. Go to Azure Portal.
  2. Create a new Event Hub Namespace.
  3. Inside the namespace, create a new Event Hub.
  4. Note down the connection string and Event Hub name.

## Task 03
#### Capture the Incoming Stream Using Structured Streaming in Databricks

In [0]:
from pyspark.sql.functions import expr, col
from pyspark.sql.types import IntegerType, StringType

Define Event Hub connection details

In [0]:
event_hub_conf = {
    'eventhubs.connectionString': 'EVENT_HUB_CONNECTION_STRING'
}

Read stream from Event Hub

In [0]:
event_stream = (spark.readStream
                .format('eventhubs')
                .options(**event_hub_conf)
                .load())

Convert binary body to integer

In [0]:
numbers_stream = event_stream.selectExpr("CAST(body AS STRING) AS number")

Convert the string to integer

In [0]:
numbers_stream = numbers_stream.withColumn("number", col("number").cast(IntegerType()))

## Task 04
#### Add 'Risk' column based on the value

In [0]:
numbers_with_risk = numbers_stream.withColumn('Risk', expr("CASE WHEN number > 80 THEN 'High' ELSE 'Low' END"))

Show the streaming data in the console

In [0]:
query = (numbers_with_risk.writeStream
         .outputMode("append")
         .format("console")
         .start())

query.awaitTermination()

## Task 05
#### Capture the Output Stream in a Separate Event Hub

In [0]:
from azure.eventhub import EventHubProducerClient, EventData

output_event_hub_conf = {
    'eventhubs.connectionString': 'OUTPUT_EVENT_HUB_CONNECTION_STRING'
}

Function to send data to output Event Hub

In [0]:
def send_to_output_event_hub(row):
    producer = EventHubProducerClient.from_connection_string(conn_str=output_event_hub_conf['eventhubs.connectionString'])
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(str(row)))
    producer.send_batch(event_data_batch)
    producer.close()

Send the processed stream to another Event Hub

In [0]:
output_query = (numbers_with_risk
                .writeStream
                .foreach(send_to_output_event_hub)
                .start())

output_query.awaitTermination()

## Task 06
#### Connect the Output Stream to Power BI Using Stream Analytics
  1. Create a Stream Analytics Job in Azure.
  2. Set the input as your output Event Hub.
  3. Set the output as Power BI:
    - Authorize Power BI.
    - Create a new dataset and table.

## Task 07
#### Build a Real-time Dashboard in Power BI
  1. Open Power BI and connect to the dataset created by your Stream Analytics job.
  2. Create a new report and add a bar chart or a table to visualize the count of High and Low risk values.
  3. Publish the report to your Power BI workspace.
  4. Pin the visuals to a dashboard to create a real-time dashboard.
  5. Write a query to count the number of High and Low risk values:

In [0]:
%sql
SELECT
    Risk,
    COUNT(*) AS Count
INTO
    PowerBIOutput
FROM
    InputStream
GROUP BY
    TumblingWindow(seconds, 2),
    Risk